In [ ]:
import glob
import nltk
import re
nltk.download('popular');
from nltk.corpus import stopwords
import urllib.request
from nltk import word_tokenize
import string
from collections import Counter
import numpy as np
from collections import OrderedDict

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
def get_boat_poems():
  poems = []
  from bs4 import BeautifulSoup
  # get poems html
  url = 'https://discoverpoetry.com/poems/poems-about-ships/'
  contents = urllib.request.urlopen(url).read()
  soup = BeautifulSoup(contents)
  for poem_html in soup.find_all('article', {'class': 'poem-listing'}):
    poem = re.search('<p class="ExcerptText">(.*?)</blockquote>', str(poem_html), re.DOTALL).groups(1)
    poems.append(poem[0])
  return poems

In [ ]:
corpus=get_boat_poems()

In [ ]:
corpus

['\r\n                                Ay, tear her tattered ensign down!<br/>\n<span class="indent-single">Long has it waved on high,</span><br/> \r\n                                And many an eye has danced to see<br/>\n<span class="indent-single">That banner in the sky;</span><br/> \r\n                                Beneath it rung the battle shout,<br/>\n<span class="indent-single">And burst the cannon’s roar;—</span><br/> \r\n                                The meteor of the ocean air<br/>\n<span class="indent-single">Shall sweep the clouds no more! </span><br/>\n</p>\n<p>\r\n                                Her deck, once red with heroes’ blood<br/>\n<span class="indent-single">Where knelt the vanquished foe,</span><br/> \r\n                                When winds were hurrying o’er the flood<br/>\n<span class="indent-single">And waves were white below,</span><br/> \r\n                                No more shall feel the victor’s tread,<br/>\n<span class="indent-single">Or k

In [ ]:
def doc_dict_func(corpus):
  c=0
  doc_dict={}
  for i in corpus:
    doc_dict[c]=i
    c+=1
  return doc_dict


In [ ]:
doc_dict=doc_dict_func(corpus)

In [ ]:
def remove_puctuation(doc_dict):
    stop_words = stopwords.words('english') 
    vocabulary = []
    for doc in doc_dict.values():
        for word in word_tokenize(doc.lower().strip()): 
            if not word in stop_words:
              vocabulary.append(word)
    return  set(vocabulary)

In [ ]:
vocabulary=remove_puctuation(doc_dict)

In [ ]:
def termfreq(vocabulary, doc_dict):
    term_freq_dict = {}
    for doc_id in doc_dict.keys():
        term_freq_dict[doc_id] = {}
    
    for word in vocabulary:
        for doc_id,doc in doc_dict.items():
            term_freq_dict[doc_id][word] = doc.count(word)
    return term_freq_dict

In [ ]:
def word_doc_freq(vocabulary, doc_dict):
    word_doc_freq_dict = {}
    for word in vocabulary:
        freq = 0
        for doc in doc_dict.values():            
           if word in word_tokenize(doc.lower().strip()):
                freq +=1
        word_doc_freq_dict[word] = freq
    return word_doc_freq_dict

In [ ]:
def inversedoc_fre(vocabulary,word_doc_freq_dict,total_doc):
    inverse_word_doc_freq_dict= {} 
    for word in vocabulary:     
        inverse_word_doc_freq_dict[word] = np.log2((total_doc+1) / word_doc_freq_dict[word])
    return inverse_word_doc_freq_dict

In [ ]:
def tfidf(vocabulary,term_freq_dict,inverse_word_doc_freq_dict,doc_dict):
    tf_idf_score_dict = {}
    for doc_id in doc_dict.keys():
        tf_idf_score_dict[doc_id] = {}
    for word in vocabulary:
        for doc_id,doc in doc_dict.items():
            tf_idf_score_dict[doc_id][word] = term_freq_dict[doc_id][word] * inverse_word_doc_freq_dict[word]
    return tf_idf_score_dict

In [ ]:
def vsm(query, doc_dict,tf_idf_score_dict,number_of_result_doc):
    query_vocab = []
    for word in query.split():
        if word not in query_vocab:
            query_vocab.append(word)

    query_wc = {}
    for word in query_vocab:
        query_wc[word] = query.lower().split().count(word)
    
    relevance_scores = {}
    for doc_id in doc_dict.keys():
        score = 0
        for word in query_vocab:
            score += query_wc[word] * tf_idf_score_dict[doc_id][word]
        relevance_scores[doc_id] = score
    sorted_value = OrderedDict(sorted(relevance_scores.items(), key=lambda x: x[1], reverse = True))
    result = {k: sorted_value[k] for k in list(sorted_value)[:number_of_result_doc]}
    result = sorted(result.items(), key=lambda x: x[1], reverse=True)
    return result

In [ ]:
vocabulary=remove_puctuation(doc_dict)
term_freq_dict=termfreq(vocabulary, doc_dict)
word_doc_freq_dict=word_doc_freq(vocabulary, doc_dict)
inverse_word_doc_freq_dict=inversedoc_fre(vocabulary,word_doc_freq_dict,len(doc_dict))
tf_idf_score_dict=tfidf(vocabulary,term_freq_dict,inverse_word_doc_freq_dict,doc_dict)


In [ ]:
result=vsm('ship wreck', doc_dict,tf_idf_score_dict,25)

In [ ]:
result

[(14, 8.320517714938909),
 (23, 8.320517714938909),
 (24, 7.446048597022769),
 (16, 6.712970633847461),
 (12, 5.471962534343666),
 (5, 4.9640323980151795),
 (13, 3.7230242985113846),
 (2, 3.3564853169237305),
 (11, 3.3564853169237305),
 (19, 3.3564853169237305),
 (6, 2.4820161990075897),
 (7, 2.4820161990075897),
 (1, 1.2410080995037949),
 (8, 1.2410080995037949),
 (10, 1.2410080995037949),
 (18, 1.2410080995037949),
 (0, 0.0),
 (3, 0.0),
 (4, 0.0),
 (9, 0.0),
 (15, 0.0),
 (17, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0)]

In [ ]:
queries = dict(enumerate([
    'ship wreck',
    'sail sailing ship',
    'little boat'
    
]))

# RELEVANCE JUDGEMENTS list with [(query_id, document_id, judgement), ...] judgement 0 | 1 with 1 = relevant
qrels = [
         (0, 9, 1),
         (0,11,1),
         (0,12,1),
         (0,13,0),
         (0,14,0),
         (0,0,0),
         (0,24,0),
         (0,17,0),
         (0,3,0),
         (0,4,0),

         (1, 0, 0),
         (1, 1, 0),
         (1, 2, 1),
         (1, 3, 1),
         (1, 5, 0),
         (1, 7, 1),
         (1, 21, 0),
         (1, 23, 1),
        
]

In [ ]:
def precision_at_k(query_id, k=10):

  doc_ranking = vsm(queries[query_id],doc_dict,tf_idf_score_dict,k)
  retrieved = [doc[0] for doc in doc_ranking[:k]] # take only the document id, rather than score

  TP = np.array([int((query_id, doc, 1) in qrels) for doc in retrieved]).sum()
  FP = np.array([int((query_id, doc, 0) in qrels) for doc in retrieved]).sum()

  precision = TP / (TP+FP)

  return TP, FP, precision

In [ ]:
def f1_score_at_k(query_id, k=10):
  # calculate f_1 score
  # hint: you need to find TP's etc in a similar way to precision at k
  doc_ranking = vsm(queries[query_id],doc_dict,tf_idf_score_dict,k)
  retrieved = [doc[0] for doc in doc_ranking[:k]] # take only the document id, rather than score
  
  TP, FP, precision = precision_at_k(query_id, k)
  relevant_docs = np.array(qrels)
  relevant_docs = relevant_docs[relevant_docs[:, 0] == query_id][:,2].sum()
  FN = relevant_docs - TP

  recall = TP / (TP+FN)
  f1 = (2 * precision * recall) / (precision + recall)
  
  return f1

In [ ]:
# To retrieve and calculate accuracy metrics for each query lets loop over them
k = 10
for query_id, query in queries.items():
  tp, fp, precision = precision_at_k(query_id, k=k)
  f1_score = f1_score_at_k(query_id, k=k)
  print('retrieved query "{}" with Precision@{} = {} and F1-score = {}'.format(query, k, precision, f1_score))


retrieved query "ship wreck" with Precision@10 = 0.4 and F1-score = 0.5
retrieved query "sail sailing ship" with Precision@10 = 0.6666666666666666 and F1-score = 0.5714285714285715
retrieved query "little boat" with Precision@10 = nan and F1-score = nan


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':
